In [10]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import sklearn
from sklearn.linear_model import LinearRegression 
import pylab as plt 
%matplotlib inline 
from sklearn import linear_model
df = pd.read_csv('TecloChurn.csv',header=0)
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [11]:
df.replace(" ", np.nan, inplace = True)
df.dropna(subset=["TotalCharges"], axis=0, inplace = True)
df.reset_index(drop = True, inplace = True)
cleanup_nums = {"PhoneService": {"Yes": 1, "No": 0}, "PaperlessBilling": {"Yes": 1, "No": 0}, "Churn": {"Yes": 1, "No": 0}}
df.replace(cleanup_nums, inplace=True)
cleanup_nums = {"MultipleLines": {"Yes": 1, "No": 0, "No phone service": 2}}
df.replace(cleanup_nums, inplace=True)
cleanup_nums = {"InternetService": {"Fiber optic": 1, "DSL": 2, "No": 0}}
df.replace(cleanup_nums, inplace=True)
cleanup_nums = {"OnlineSecurity": {"Yes": 1, "No": 0, "No internet service": 2}, "DeviceProtection": {"Yes": 1, "No": 0, "No internet service": 2}, "TechSupport": {"Yes": 1, "No": 0, "No internet service": 2}, "StreamingTV": {"Yes": 1, "No": 0, "No internet service": 2}, "StreamingMovies": {"Yes": 1, "No": 0, "No internet service": 2}, "Contract": {"Month-to-month": 1, "Two year": 3, "One year": 2}, "PaymentMethod": {"Electronic check": 1, "Mailed check": 2, "Bank transfer (automatic)": 3, "Credit card (automatic)": 4}}
df.replace(cleanup_nums, inplace=True)
cleanup_nums = {"OnlineBackup": {"Yes": 1, "No": 0, "No internet service": 2}}
df.replace(cleanup_nums, inplace=True)
cleanup_nums = {"gender": {"Male": 1, "Female": 0}, "Partner": {"Yes": 1, "No": 0}, "Dependents": {"Yes": 1, "No": 0}}
df.replace(cleanup_nums, inplace=True)

In [12]:
df = df.drop('customerID', axis = 1)


In [13]:
X = np.asarray(df[['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', "InternetService", "OnlineSecurity", "OnlineBackup", "DeviceProtection", "TechSupport", "StreamingTV", "StreamingMovies", "Contract", "PaperlessBilling", "PaymentMethod", "MonthlyCharges", "TotalCharges"]])


In [15]:
Y = np.asarray(df['Churn'])


In [17]:
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split( X, Y, test_size=0.2, random_state=4)

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
LR = LogisticRegression(C=0.01, solver='liblinear').fit(X_train,Y_train)

In [20]:
yhat = LR.predict(X_test)
yhat_prob = LR.predict_proba(X_test)

In [23]:
yhat_prob #En la primer columna esa la probabilidad de que sea 0, y en la segunda la que sea 1

array([[0.87519838, 0.12480162],
       [0.73033803, 0.26966197],
       [0.9482254 , 0.0517746 ],
       ...,
       [0.96919214, 0.03080786],
       [0.41714802, 0.58285198],
       [0.36411497, 0.63588503]])

Nos interesa la probabilidad de que sea 1 (alguien se fue)

In [26]:
prob_churn = yhat_prob[:,1]

Ahora preguntamos que valores estan entre 0.4 y 0.6

In [42]:
indexes_churn =  [x for x in range(len(prob_churn)) if (prob_churn[x] > 0.4) and (prob_churn[x] < 0.6)]

Eso nos da los indices